## VaR implementation

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

# Download the data
amzn = yf.download('AMZN', '2023-11-30', '2024-11-30')
tsla = yf.download('TSLA', '2023-11-30', '2024-11-30')
aapl = yf.download('AAPL', '2023-11-30', '2024-11-30')

C:\Users\Admin\AppData\Local\Temp\ipykernel_2216\4119724011.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  amzn = yf.download('AMZN', '2023-11-30', '2024-11-30')
[*********************100%***********************]  1 of 1 completed
C:\Users\Admin\AppData\Local\Temp\ipykernel_2216\4119724011.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tsla = yf.download('TSLA', '2023-11-30', '2024-11-30')
[*********************100%***********************]  1 of 1 completed
C:\Users\Admin\AppData\Local\Temp\ipykernel_2216\4119724011.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  aapl = yf.download('AAPL', '2023-11-30', '2024-11-30')
[*********************100%***********************]  1 of 1 completed


In [2]:
# Extract only closing prices
amzn_close = amzn['Close']
tsla_close = tsla['Close']
aapl_close = aapl['Close']

# Put all three closing prices together
df = pd.concat([amzn_close, tsla_close, aapl_close], axis=1)
df.columns = ['AMZN', 'TSLA', 'AAPL']

# Compute the returns
df['R1'] = -df['AMZN'].pct_change()
df['R2'] = -df['TSLA'].pct_change()
df['R3'] = -df['AAPL'].pct_change()

# Construct the portfolio returns column as a weighted sum of individual asset returns and weights
w = np.array([0.4, 0.3, 0.3])
df['Rp'] = (df[['R1', 'R2', 'R3']] * w).sum(axis=1)
df = df.drop(df.index[0])
df

,AMZN,TSLA,AAPL,R1,R2,R3,Rp
Date,,,,,,,
2023-12-01,147.029999,238.830002,189.849701,-0.006434,0.005207,-0.006791,-0.003049
2023-12-04,144.839996,235.580002,188.052856,0.014895,0.013608,0.009465,0.012880
2023-12-05,146.880005,238.720001,192.013855,-0.014085,-0.013329,-0.021063,-0.015951
2023-12-06,144.520004,239.369995,190.921890,0.016068,-0.002723,0.005687,0.007316
2023-12-07,146.880005,242.639999,192.857681,-0.016330,-0.013661,-0.010139,-0.013672
...,...,...,...,...,...,...,...
2024-11-22,197.119995,352.559998,229.316818,0.006351,-0.038040,-0.005908,-0.010644
2024-11-25,201.449997,338.589996,232.309601,-0.021966,0.039624,-0.013051,-0.000814
2024-11-26,207.860001,338.230011,234.494324,-0.031819,0.001063,-0.009404,-0.015230


## Non-parametric VaR

In [3]:
# Define initial portfolio value and add a column of portfolio values to the df object
V0 = 1000000
df['V'] = V0 * (1 + df['Rp']).cumprod()

# Compute and report portfolio VaR(mean) and VaR(zero)
print('VaR(mean) is ${:.2f}'.format(df['V'].mean() - df['V'].quantile(0.05)))
print('VaR(zero) is ${:.2f}'.format(V0 - df['V'].quantile(0.05)))

VaR(mean) is $230540.84
VaR(zero) is $325539.40


## Parametric VaR  

In [4]:
# Define initial portfolio value and add a column of portfolio values to the df object
V0 = 1_000_000
df['V'] = V0 * (1 + df['Rp']).cumprod()

z = norm.ppf(0.95)

# Compute VaR(mean) assuming mean return is 0
print('VaR(mean) is ${:.2f}'.format(V0 * z * df['Rp'].std() * np.sqrt(252)))
# Compute VaR(zero) accounting for mean return
print('VaR(zero) is ${:.2f}'.format(V0 * (z * df['Rp'].std() * np.sqrt(252) - df['Rp'].mean() * np.sqrt(252))))

VaR(mean) is $450631.49
VaR(zero) is $475676.55


## Portfolio VaR

In [5]:
# Compute portfolio variance and standard deviation
variance_p = df['Rp'].var() * 252
sigma_p = np.sqrt(variance_p)

# Define initial portfolio value and a standard normal multiple for 95% confidence level
W = 1000000
z = norm.ppf(0.95)

# Compute and report portfolio VaR
VaR_p = sigma_p * z * W
print('Portfolio VaR corresponding to 0.95 confidence level is ${:.2f}'.format(VaR_p))

Portfolio VaR corresponding to 0.95 confidence level is $450631.49


## Margin VaR

In [6]:
# Compute individual asset betas, express them as a vector
beta_1 = df[['R1', 'Rp']].cov().iloc[0, 1] * 252 / variance_p
beta_2 = df[['R2', 'Rp']].cov().iloc[0, 1] * 252 / variance_p
beta_3 = df[['R3', 'Rp']].cov().iloc[0, 1] * 252 / variance_p
beta = np.array([beta_1, beta_2, beta_3])

# Compute Marginal VaRs as a vector
MVaR = z * sigma_p * beta
print('Marginal VaRs for the assets respectively are: ' + ', '.join([f'${value:,.2f}' for value in MVaR]))

Marginal VaRs for the assets respectively are: $0.30, $0.87, $0.22


## Incremental VaR  

In [7]:
# Define positions change vector and compute new portfolio VaR
a = np.array([10000, 5000, 0.0])
VaR_new = z * sigma_p * (W + sum(a))

# Compute Incremental VaR (it is a single number)
delta_VaR = VaR_new - VaR_p
print(f'Incremental VaR caused by the new position vector a is: ${delta_VaR:,.2f}')

Incremental VaR caused by the new position vector a is: $6,759.47


## Component VaR

In [8]:
# Compute Component VaRs as a vector
CVaR = VaR_p * beta * w
print(f"Component VaRs for the assets respectively are: {', '.join([f'${x:,.2f}' for x in CVaR])}")

Component VaRs for the assets respectively are: $121,851.09, $262,084.08, $66,696.33


## Expected Shortfall

In [9]:
# Add the Portfolio column with an initial value of 1,000,000
W = 1000000
df['Portfolio'] = W * (1 + df['Rp']).cumprod()

# Compute losses and add as a column to df
df['L'] = W - df['Portfolio']

# Compute VaR(zero)
VaR_zero = W - df['Portfolio'].quantile(0.05)

# Expected Shortfall
# ES = df.loc[df['L'] > VaR_zero, 'L'].mean()
# print('Expected Shortfall is: {:.2f}'.format(ES))

ES_analytic = df['L'].mean() + df['L'].std() * norm.pdf(norm.ppf(0.95)) / (1 - 0.95)
print('Expected Shortfall is: {:.2f}'.format(ES_analytic))

Expected Shortfall is: 327669.07
